In [1]:
from src.visualisation import instance_map_visualization
from src.model.perception.labeler import LabelGenerator
import os
import pathlib
import numpy as np
from src.config import default_map_builder_cfg, default_map_processor_cfg, default_sensor_cfg
from src.visualisation.semantic_map_visualization import visualize_categorical_label_map
import _pickle as cPickle
import open3d as o3d
from src.utils import category_mapping

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
if pathlib.PurePath(os.getcwd()).name == "notebooks":
    print(pathlib.PurePath(os.getcwd()).name)
    os.chdir("..")


notebooks


In [3]:
epoch = 0
id = "00041-QKfBMSSy7Hy"
map_builder_path = f"data/interim/trajectories/epoch_{epoch}/{id}/map_builder.pickle"
with open(map_builder_path, 'rb') as fp:
    map_builder = cPickle.load(fp)

In [4]:
plot_point_cloud = True

if plot_point_cloud:
    point_cloud = map_builder.point_cloud
    point_cloud_semantic_labels = map_builder._master_point_cloud_semantic_labels

    color_converter = category_mapping.get_reseal_index_to_color_vectorized()

    point_cloud_categorical = np.argmax(point_cloud_semantic_labels, axis=-1) + (np.sum(
        point_cloud_semantic_labels, axis=-1
    ) > 0).astype(int)
    # Need to initialize this as a 3 channel image so we can use numpy indexing to assign the points that have a label
    # instead of having to loop over them
    point_cloud_rgb = np.dstack([point_cloud_categorical] * 3)[0]
    #point_cloud_rgb[point_cloud_categorical > 0] = color_converter(point_cloud_categorical[point_cloud_categorical > 0])

    point_cloud_o3d = o3d.utility.Vector3dVector(point_cloud[point_cloud_categorical > 0])
    pcd = o3d.geometry.PointCloud(point_cloud_o3d)
    colour = color_converter(point_cloud_categorical[point_cloud_categorical > 0])
    pcd.colors = o3d.utility.Vector3dVector(colour)
    o3d.visualization.draw_geometries([pcd])

In [5]:
label_generator = LabelGenerator(map_builder.semantic_map, map_builder.get_grid_index_of_origin(), default_map_builder_cfg(), default_map_processor_cfg(), default_sensor_cfg())

In [6]:
plotter = visualize_categorical_label_map(label_generator.categorical_label_map, opacity=0.1)

# If nothing shows up, try changing jupyter_backend="trame" to jupyter_backend="panel"
plotter.show()

Widget(value="<iframe src='http://localhost:43555/index.html?ui=P_0x7fb3b8df1f10_0&reconnect=auto' style='widt…